In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import math
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
def chooseModel(province,inputshape):
    if(province=='BKK'):
        modelLSTM_BKK = Sequential()

        modelLSTM_BKK.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_BKK.add(Dropout(0.2))
        modelLSTM_BKK.add(LSTM(units = 80, activation = 'relu' ,return_sequences = True))
        modelLSTM_BKK.add(Dropout(0.2))
        modelLSTM_BKK.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_BKK.add(Dropout(0.2))


        modelLSTM_BKK.add(Dense(units = 12))
        return modelLSTM_BKK
    if(province=='Chiangmai'):
        modelLSTM_Chiangmai = Sequential()

        modelLSTM_Chiangmai.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Chiangmai.add(Dropout(0.2))
        modelLSTM_Chiangmai.add(LSTM(units = 80, activation = 'relu'))
        modelLSTM_Chiangmai.add(Dropout(0.2))


        modelLSTM_Chiangmai.add(Dense(units = 12))
        return modelLSTM_Chiangmai
    if(province=='Khonkaen'):
        modelLSTM = Sequential()

        modelLSTM.add(LSTM(units = 100, activation = 'relu', input_shape = (inputshape, 4)))
        modelLSTM.add(Dropout(0.2))


        modelLSTM.add(Dense(units = 12))
        return modelLSTM
    if(province=='Rayong'):
        modelLSTM_Rayong = Sequential()

        modelLSTM_Rayong.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Rayong.add(Dropout(0.2))

        modelLSTM_Rayong.add(LSTM(units = 120, activation = 'relu'))
        modelLSTM_Rayong.add(Dropout(0.2))


        modelLSTM_Rayong.add(Dense(units = 12))
        return modelLSTM_Rayong

    if(province=='Saraburi'):
        modelLSTM_Saraburi = Sequential()

        modelLSTM_Saraburi.add(LSTM(units = 120, activation = 'relu',  input_shape = (inputshape, 4)))
        modelLSTM_Saraburi.add(Dropout(0.2))

        modelLSTM_Saraburi.add(Dense(units = 12))
        return modelLSTM_Saraburi
    if(province=='Surat'):
        modelLSTM_Surat = Sequential()

        modelLSTM_Surat.add(LSTM(units = 120, activation = 'relu', return_sequences = True, input_shape = (inputshape, 4)))
        modelLSTM_Surat.add(Dropout(0.2))
        modelLSTM_Surat.add(LSTM(units = 240, activation = 'relu'))
        modelLSTM_Surat.add(Dropout(0.2))



        modelLSTM_Surat.add(Dense(units = 12))

        return modelLSTM_Surat

In [3]:
all_config ={'BKK':{
                    'batch': 32,
                    'epoch': 10,
},
'Chiangmai':{
                    'batch': 32,
                    'epoch': 10,
},
'Khonkaen':{
                    'batch': 8,
                    'epoch': 10,
}
,'Rayong':{
                    'batch': 32,
                    'epoch': 10,
}
,'Saraburi':{
                    'batch': 32,
                    'epoch': 10,
},'Surat':{
                    'batch': 64,
                    'epoch': 15,
},}

In [6]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']

for province in provinces:
    print(province)
    data_training = pd.read_csv('../'+province+'/train/'+province.lower()+'_train_format.csv', date_parser = True)

    data_training = data_training.drop(['date_time', 'lat', 'long'], axis = 1)

    data_test = pd.read_csv(province+'_clean2.csv', date_parser = True)

    scaler = MinMaxScaler()
    data_training = scaler.fit_transform(data_training)
    scaler.inverse_transform(data_training)
    scale = 1/scaler.scale_

    X_train = []
    y_train = []

    for i in range(60, data_training.shape[0]):
        X_train.append(data_training[i-60:i])
        a = []
        for j in range(1,13):
            #print(i,j,i-60+j*6)
            a.append(data_training[min(i-60+j*6,data_training.shape[0]-1),0])
        y_train.append(a)


    X_train, y_train = np.array(X_train), np.array(y_train)



        #create callback
    filepath = province+'_modelLSTM.hdf5'
    checkpoint = ModelCheckpoint(filepath=filepath, 
                                monitor='loss',
                                verbose=0, 
                                save_best_only=True,
                                mode='min')

    earlystopping = EarlyStopping(
        monitor='loss', 
        patience=3, 
        min_delta=0, 
        mode='auto'
    )

    reduce_lr = ReduceLROnPlateau(
        monitor='loss', 
        factor=0.2,   
        patience=2, 
        min_lr=0.001,
        verbose=0
    )
    callbacks = [checkpoint, earlystopping, reduce_lr]

    modelLSTM = chooseModel(province,X_train.shape[1])
    modelLSTM.compile(optimizer='adam', loss = 'mean_squared_error')
    history = modelLSTM.fit(X_train, y_train, epochs=all_config[province]['epoch'], batch_size=all_config[province]['batch'], callbacks=callbacks)

    data_training = pd.read_csv('../'+province+'/train/'+province.lower()+'_train_format.csv', date_parser = True)
    data_training = data_training.drop(['date_time', 'lat', 'long'], axis = 1)
    data_test = data_test.drop(['date_time'], axis = 1)

    past_60_days = data_training.tail(60)
    df = past_60_days.append(data_test, ignore_index = True)
    inputs = scaler.transform(df)
    inputs = scaler.transform(past_60_days)
    X_test = []
    ans_y_pred = []
    scale = 1/scaler.scale_[0]
    # print("inputs",inputs)

    X_test.append(np.array(inputs))
    X_test = np.array(X_test)

    # print(X_test)

    y_pred = modelLSTM.predict(X_test)

    # print("y_pred",y_pred)

    ans_y_pred.append(y_pred)


    a = np.array(data_test)

    for i in range(1, data_test.shape[0]):
        # print(y_pred[0][0])
        inputs = scaler.transform(np.array([[y_pred[0][0]*scale,a[i][1],a[i][2],a[i][3]]]))
        
        # print("inputs",inputs)

        X_test = np.append(X_test,np.array([inputs]),axis=1)
        # print("X_test -60",X_test[:,-60:,:].shape)
        # print(X_test)

        y_pred = modelLSTM.predict([X_test[:,-60:,:]])
        ans_y_pred.append(y_pred)
    
    ans_y_pred = np.array(ans_y_pred)
    ans_y_pred = ans_y_pred*scale

    df2 = []
    for x in ans_y_pred:
        # print(x)
        a = []
        for i in range(0,12):
            a.append(x[0][i])
        df2.append(a)
    df2 = pd.DataFrame(df2)
    df2.to_csv(province+'_pred.csv',index=False)

    with open('lstm_'+province.lower()+'.pickle', 'wb') as fp:
        pickle.dump(y_pred, fp)

    


BKK
Epoch 1/10
460/821 [===============>..............] - ETA: 1:01 - loss: 0.0075

KeyboardInterrupt: 